In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('../Health.csv')
df.shape

C:\Users\MichaLaptop\AppData\Local\Temp\ipykernel_14592\1728526612.py:1: DtypeWarning: Columns (3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../Health.csv')


(170761, 42)

In [4]:
# тут я выводил количество nan, оказались они только в 1 столбце(верьте мне)
df.isna().sum();

In [16]:
df.duplicated().sum()

0

In [6]:
df[df['sagey_b'].isna()]

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rjcten,index_wave,insured_gov,uninsured,retired,collegeplus,year,cpi,rgov,sgov
40359,42633021,1,1,0,3,15apr2006,53,NaN,3,1,...,.w,8,1,0,0,0,2005,1.133913,4680.0,10032.0
48942,48544011,1,1,0,1,15oct2006,46,NaN,3,1,...,1.9,8,0,0,0,0,2005,1.133913,0.0,0.0
85784,85066011,1,1,0,2,15apr2006,68,NaN,3,0,...,.w,8,1,0,100,0,2005,1.133913,7476.0,276.0
149436,500893010,3,2,0,1,15may2006,54,NaN,4,0,...,26.7,2,0,0,0,1,2005,1.133913,0.0,0.0


In [9]:
df['sagey_b'].fillna('.r', inplace=True) # будем считать, что те 4 человека отказались от ответа

In [14]:
import datetime

months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

def parse_date(el):
    day, month, year = el[:2], el[2:5], el[5:]

    return datetime.date(int(year), months.index(month) + 1, int(day))


df['riwbegy'] = df['riwbegy'].apply(parse_date)

In [15]:
df.head()

,hhidpn,rmstat,ragender,rahispan,raracem,riwbegy,ragey_b,sagey_b,rhltc,rhlthlm,...,rjcten,index_wave,insured_gov,uninsured,retired,collegeplus,year,cpi,rgov,sgov
0,2010,7,2,0,1,1998-06-15,63,.u,4,1,...,.w,4,1,0,100,0,1997,0.934777,7394.6318,0.0
1,2010,7,2,0,1,1996-10-15,62,.u,3,1,...,.w,3,1,0,100,0,1995,0.891173,7374.7271,0.0
2,2010,7,2,0,1,2000-03-15,65,.u,3,1,...,.w,5,1,0,100,0,1999,0.967805,7029.8052,0.0
3,2010,7,2,0,1,1994-07-23,59,.u,3,1,...,.1,2,1,0,0,0,1993,0.852219,4023.5574,0.0
4,2010,7,2,0,1,1992-09-18,57,.u,3,1,...,.w,1,1,0,100,0,1991,0.811126,4193.8506,0.0


In [26]:
# надо будет подумать, являются ли эти данные выбросами(скорее нет)
df['sgov'].describe()

count    170761.000000
mean       3856.876258
std        7071.853159
min           0.000000
25%           0.000000
50%           0.000000
75%        6492.137200
max      159528.700000
Name: sgov, dtype: float64

In [21]:
df['rgov'].describe()

count    170761.000000
mean       6545.835385
std        7697.727228
min           0.000000
25%           0.000000
50%        5208.233900
75%       11346.187000
max      112269.450000
Name: rgov, dtype: float64

In [27]:
# а тут меня очень смущает максимум в столбце sipena, мб это выброс
df.iloc[:, 21:25].describe()

,riearn,ripena,siearn,sipena
count,170761.000000,170761.000000,170761.000000,1.707610e+05
mean,15702.244733,3764.862189,12042.235232,2.579190e+03
std,33371.000325,12092.210896,30816.336503,2.217670e+04
min,0.000000,0.000000,0.000000,0.000000e+00
25%,0.000000,0.000000,0.000000,0.000000e+00
50%,0.000000,0.000000,0.000000,0.000000e+00
75%,21089.415692,0.000000,8000.000000,0.000000e+00
max,643233.038611,319329.942983,615098.104813,6.780589e+06
